# Загрузка данных + импорты

## Импорты

In [461]:
#импорты
#from nltk.tokenize import word_tokenize, RegexpTokenizer
#from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.naive_bayes import ComplementNB
#from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from catboost import CatBoostClassifier


In [462]:
FILESOURCE = 'SMSSpamCollection.txt' # файл источник


## Чтение файла

In [463]:
# читаем файл
with open(FILESOURCE) as f: 
    full_file = f.read()
print(full_file[:20])

ham	Go until jurong 


**Вывод:** данные удалось прочитать, уже неплохо

# Предобработка

In [464]:
# нижний регистр и разделение на строки.
full_file = full_file.lower()
lst = full_file.splitlines()

In [465]:
# Отделение таргета
lst_with_target = []
for line in lst:
    lst_tmp = []
    lst_line = line.split('\t')  
    if lst_line[0] == 'spam' or lst_line[0] == 'ham':
        lst_tmp.append('ok')
        lst_tmp.append(lst_line[0])
        lst_tmp.append(lst_line[1:])
        lst_with_target.append(lst_tmp)
    else:
        lst_tmp.append('not_ok')
        lst_tmp.append(lst_line[0])
        lst_tmp.append(lst_line[1:])
        lst_with_target.append(lst_tmp)

In [466]:
# проверка на структуру файла
for i in range(len(lst_with_target)):
    if lst_with_target[i][0] != 'ok':
        print(lst_with_target[i], 'is not ok')
    if len(lst_with_target[i]) > 3:
        print(lst_with_target[i], '>3')

**Вывод:** структура файла соблюдается

In [467]:
vectorizer = CountVectorizer(stop_words='english', token_pattern='[a-z]\w+')
X = vectorizer.fit_transform(lst)
#vectorizer.get_feature_names_out()


In [468]:
vectorizer2 = CountVectorizer(token_pattern='[0-9]\w+')
X2 = vectorizer2.fit_transform(lst)
vectorizer3 = CountVectorizer(token_pattern='[§!@#$%^&*()_+-<>:"|\?~{}]')
X3 = vectorizer3.fit_transform(lst)


In [469]:
df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names_out())
df2 = pd.DataFrame(X2.toarray(), columns= vectorizer2.get_feature_names_out())
df3 = pd.DataFrame(X3.toarray(),columns=vectorizer3.get_feature_names_out())

In [470]:
df['words_with_digits'] = df2.T.sum()
df['special_symbols'] = df3.T.sum()

In [471]:
df['target'] =  (df['spam'] > 0).apply(int)

df = df.drop(['spam', 'ham'], axis=1)

In [472]:
len(vectorizer.get_feature_names_out())

7680

In [473]:
# Создаем транспонированый df
df_t = df.T

## Среднее количество слов в сообщении

In [474]:
df[df['target'] == 1].T.sum().mean()

39.163319946452475

In [475]:
df[df['target'] == 0].T.sum().mean()

10.946550652579242

In [476]:
df['len_diff'] = abs(df_t.sum() - features_train[target_train == 1].T.sum().mean())

In [477]:
# разделение
features = df.drop('target',axis=1)
target = df['target']
features_train, features_test, target_train, target_test = train_test_split(features,target, test_size=.3, random_state=123456)
print(features_train.shape, features_test.shape, target_train.shape, target_test.shape )

(3901, 7681) (1673, 7681) (3901,) (1673,)


### Сбалансированный df

In [478]:
features_train_ones = features_train[target_train == 1]

In [479]:
features_train_zeros = features_train[target_train == 0].sample(n=features_train_ones.shape[0])

In [480]:
features_train_balanced = pd.concat([features_train_ones, features_train_zeros])

In [481]:
features_train_balanced = features_train_balanced.join(target_train)
features_train_balanced = shuffle(features_train_balanced)

In [482]:
target_train_balanced = features_train_balanced['target']

In [483]:
features_train_balanced = features_train_balanced.drop('target', axis=1)

# EDA

## Баланс классов

### Общий баланс классов

In [484]:
df['target'].mean()

0.1340150699677072

### Баланс классов train

In [485]:
target_train.mean()

0.12971033068443988

In [486]:
target_train_balanced.mean()

0.5

### Баланс классов test

In [487]:
target_test.mean()

0.14405260011954574

## Поиск важных слов


In [488]:
df[df['target'] == 1].sum().sort_values(ascending=False)[:25]

special_symbols      15925.000000
len_diff             10505.159669
words_with_digits     2540.000000
target                 747.000000
free                   226.000000
txt                    166.000000
ur                     144.000000
mobile                 128.000000
text                   125.000000
stop                   125.000000
claim                  113.000000
reply                  104.000000
www                     98.000000
prize                   93.000000
just                    79.000000
cash                    76.000000
won                     76.000000
uk                      74.000000
send                    71.000000
new                     69.000000
nokia                   67.000000
win                     64.000000
urgent                  63.000000
tone                    60.000000
week                    60.000000
dtype: float64

In [489]:
df[df['target'] == 0].sum().sort_values(ascending=False)[:25]

len_diff             193982.459026
special_symbols       18803.000000
words_with_digits       356.000000
gt                      318.000000
lt                      316.000000
just                    297.000000
ok                      288.000000
ll                      266.000000
ur                      247.000000
know                    237.000000
good                    235.000000
like                    234.000000
got                     234.000000
come                    230.000000
day                     217.000000
love                    205.000000
time                    202.000000
going                   169.000000
want                    165.000000
home                    165.000000
lor                     162.000000
don                     159.000000
need                    159.000000
sorry                   157.000000
da                      150.000000
dtype: float64

**Вывод:** Можно утверждать, что в spam-сообщениях часто употребляются такие слова как free и txt, а также есть много слов с цифрами

## Модели

In [490]:
cnb = ComplementNB()
cnb_bal = ComplementNB()
lgr = LogisticRegression(penalty='l2')
cbc = CatBoostClassifier()

In [491]:
cnb.fit(features_train,target_train)
cnb_bal.fit(features_train_balanced, target_train_balanced)
lgr.fit(features_train_balanced, target_train_balanced)
cbc.fit(features_train_balanced, target_train_balanced)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Learning rate set to 0.010355
0:	learn: 0.6820584	total: 7.28ms	remaining: 7.27s
1:	learn: 0.6692273	total: 11.8ms	remaining: 5.88s
2:	learn: 0.6587581	total: 15.6ms	remaining: 5.19s
3:	learn: 0.6480741	total: 19.9ms	remaining: 4.95s
4:	learn: 0.6350219	total: 24.1ms	remaining: 4.79s
5:	learn: 0.6224713	total: 28.4ms	remaining: 4.7s
6:	learn: 0.6105149	total: 33.3ms	remaining: 4.72s
7:	learn: 0.5977690	total: 37.7ms	remaining: 4.67s
8:	learn: 0.5863705	total: 42ms	remaining: 4.63s
9:	learn: 0.5759470	total: 47.4ms	remaining: 4.69s
10:	learn: 0.5677666	total: 49.5ms	remaining: 4.45s
11:	learn: 0.5579589	total: 54.2ms	remaining: 4.46s
12:	learn: 0.5484266	total: 58.2ms	remaining: 4.42s
13:	learn: 0.5393803	total: 63.4ms	remaining: 4.47s
14:	learn: 0.5302634	total: 67.7ms	remaining: 4.44s
15:	learn: 0.5205200	total: 71.8ms	remaining: 4.41s
16:	learn: 0.5134572	total: 76.4ms	remaining: 4.42s
17:	learn: 0.5047490	total: 80.6ms	remaining: 4.4s
18:	learn: 0.4964459	total: 84.8ms	remaining: 4.

In [492]:
def print_model_rep(model_kind, model, features_train, target_train, features_test, target_test):
    print(model_kind)
    print('train')
    prediction = model.predict(features_train)
    print(classification_report(target_train, prediction))
    print('test')
    prediction = model.predict(features_test)
    print(classification_report(target_test, prediction))



In [493]:
print_model_rep('NB not balanced', cnb, features_train, target_train, features_test, target_test)

NB not balanced
train
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3395
           1       0.86      0.91      0.88       506

    accuracy                           0.97      3901
   macro avg       0.92      0.94      0.93      3901
weighted avg       0.97      0.97      0.97      3901

test
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1432
           1       0.90      0.88      0.89       241

    accuracy                           0.97      1673
   macro avg       0.94      0.93      0.94      1673
weighted avg       0.97      0.97      0.97      1673



In [494]:
print_model_rep('NB balanced', cnb_bal, features_train_balanced, target_train_balanced, features_test, target_test)

NB balanced
train
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       506
           1       0.98      0.91      0.94       506

    accuracy                           0.95      1012
   macro avg       0.95      0.95      0.95      1012
weighted avg       0.95      0.95      0.95      1012

test
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1432
           1       0.90      0.87      0.89       241

    accuracy                           0.97      1673
   macro avg       0.94      0.93      0.93      1673
weighted avg       0.97      0.97      0.97      1673



In [495]:
print_model_rep('log_reg_balanced', lgr, features_train_balanced, target_train_balanced, features_test, target_test)


log_reg_balanced
train
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       506
           1       1.00      0.99      1.00       506

    accuracy                           1.00      1012
   macro avg       1.00      1.00      1.00      1012
weighted avg       1.00      1.00      1.00      1012

test
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1432
           1       0.91      0.94      0.93       241

    accuracy                           0.98      1673
   macro avg       0.95      0.96      0.96      1673
weighted avg       0.98      0.98      0.98      1673



In [496]:
print_model_rep('catboost', cbc, features_train_balanced, target_train_balanced, features_test, target_test)


catboost
train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       506
           1       1.00      1.00      1.00       506

    accuracy                           1.00      1012
   macro avg       1.00      1.00      1.00      1012
weighted avg       1.00      1.00      1.00      1012

test
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1432
           1       0.82      0.94      0.87       241

    accuracy                           0.96      1673
   macro avg       0.90      0.95      0.93      1673
weighted avg       0.96      0.96      0.96      1673



**Вывод** нам удалось сделать модель, которая в некоторой степени умеет классифицировать сообщения на SPAM и HAM